In [3]:
from pathlib import Path
import numpy as np
import pandas as pd
from functools import partial

import torch
import torch.nn as nn

import wiki_dataset

import importlib
importlib.reload(wiki_dataset)


from preprocess import pad_collate_fn

import training
importlib.reload(training)
from training import get_train_val_loader, ClassifierLearner

import model

from model import FinalModel

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [5]:
(index_to_word, word_to_index,
     dict_wiki_tensor_dataset,
     weights_matrix_ve, classes) = wiki_dataset.get_mixed_datasets(LANGUAGES_LIST=["english", "russian", "hindi"])

english vocab size is: 741334
russian vocab size is: 858845
hindi vocab size is: 441314
Order: dict_keys(['english', 'russian', 'hindi'])


  1%|          | 151/30000 [00:00<00:19, 1497.52it/s]

Combined train size: 30000 
Combined val size: 3000


100%|██████████| 3000/3000 [00:00<00:00, 3615.83it/s]


Embeddings matrix shape: torch.Size([2041495, 300]), 
Vocab size: 2041495


In [6]:
dict_wiki_tensor_dataset.keys()

dict_keys(['monolingual_train_en', 'multilingual_train_en', 'val_en', 'monolingual_train_ru', 'multilingual_train_ru', 'val_ru', 'monolingual_train_hi', 'multilingual_train_hi', 'val_hi', 'train', 'val'])

In [10]:
SAVE_MODEL = False

batch_size = 8
lr = 0.01
num_epochs = 15

options = {
    "VOCAB_SIZE": len(index_to_word),
    "dim_e": weights_matrix_ve.shape[1],
    "pretrained_embeddings": weights_matrix_ve,
    "num_layers": 2,
    "num_classes": len(classes),
    "mid_features": 150,
    "dropout_rate": 0.2,
    "activation": nn.ReLU(),
}
    
result = {
    "optimizer": "Adam", 
    "num_hidden": options["num_layers"],
    "dim_hidden": options["mid_features"],
    "dropout_rate": options["dropout_rate"],
    "learning_rate": lr,
    "num_epochs": num_epochs
}


print("\n", result)
model_name = "mixed_en_hi_ru_" + "_".join([str(key) + "_" + str(value) for key, value in result.items()])

learner = ClassifierLearner(options, model_name, device=device)


val_keys = ["val_en", "val_ru", "val_hi", "val"]
train_loader, dict_val_loader = get_train_val_loader(
    dict_wiki_tensor_dataset["train"], 
    [dict_wiki_tensor_dataset[key] for key in val_keys], 
    collate_fn=partial(pad_collate_fn, pad_token=word_to_index["<pad>"])
)


 {'optimizer': 'Adam', 'num_hidden': 2, 'dim_hidden': 150, 'dropout_rate': 0.2, 'learning_rate': 0.01, 'num_epochs': 15}


In [11]:
dict_val_loader = {
   key : val_loader 
   for key, val_loader in zip(val_keys, dict_val_loader)
}

In [12]:
learner.set_loaders(train_loader, dict_val_loader)

In [13]:
learner.train_model(num_epochs=10, lr=0.01)

0 epoch


/scratch/mz2476/miniconda3/envs/my_base/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision macro: 0.5616, Recall macro: 0.3511, F1 macro: 0.4065 
Precision micro: 0.8054, Recall micro: 0.5656, F1 micro: 0.6646 
1 epoch
Precision macro: 0.6751, Recall macro: 0.4115, F1 macro: 0.478 
Precision micro: 0.8034, Recall micro: 0.6141, F1 micro: 0.6961 
2 epoch
Precision macro: 0.7063, Recall macro: 0.4547, F1 macro: 0.5222 
Precision micro: 0.8079, Recall micro: 0.6146, F1 micro: 0.6981 
3 epoch
Precision macro: 0.7177, Recall macro: 0.482, F1 macro: 0.5516 
Precision micro: 0.8094, Recall micro: 0.6348, F1 micro: 0.7116 
4 epoch
Precision macro: 0.7161, Recall macro: 0.4426, F1 macro: 0.5251 
Precision micro: 0.8354, Recall micro: 0.597, F1 micro: 0.6963 
5 epoch
Precision macro: 0.6904, Recall macro: 0.4862, F1 macro: 0.5498 
Precision micro: 0.7993, Recall micro: 0.6445, F1 micro: 0.7136 
6 epoch
Precision macro: 0.7285, Recall macro: 0.5163, F1 macro: 0.5818 
Precision micro: 0.8188, Recall micro: 0.6367, F1 micro: 0.7164 
7 epoch
Precision macro: 0.7588, Recall macro

({'precision_macro': 0.7235292772710287,
  'recall_macro': 0.548840485433408,
  'f1_macro': 0.6025189506395687,
  'precision_micro': 0.7964105440269209,
  'recall_micro': 0.6745843230403801,
  'f1_micro': 0.7304526748971193},
 9)

In [71]:
import results_analysis
importlib.reload(results_analysis)
from results_analysis import plot_errorbars_by_model, get_mean_std_k

In [28]:
# model_to_mean_std = {}

In [120]:
dict_wiki_tensor_dataset

{'monolingual_train_en': return TextData(self.input_tensors[idx], self.input_len[idx], self.target_tensors[idx]),
 'multilingual_train_en': return TextData(self.input_tensors[idx], self.input_len[idx], self.target_tensors[idx]),
 'val_en': return TextData(self.input_tensors[idx], self.input_len[idx], self.target_tensors[idx]),
 'monolingual_train_ru': return TextData(self.input_tensors[idx], self.input_len[idx], self.target_tensors[idx]),
 'multilingual_train_ru': return TextData(self.input_tensors[idx], self.input_len[idx], self.target_tensors[idx]),
 'val_ru': return TextData(self.input_tensors[idx], self.input_len[idx], self.target_tensors[idx]),
 'train': return TextData(self.input_tensors[idx], self.input_len[idx], self.target_tensors[idx]),
 'val': return TextData(self.input_tensors[idx], self.input_len[idx], self.target_tensors[idx])}

In [119]:
model_name = "en_ru"
metric_name='f1_micro'

model_to_mean_std[model_name] = get_mean_std_k(
    learner, num_splits=5, dict_wiki_tensor_dataset=dict_wiki_tensor_dataset,
    metric_name=metric_name)

path_to_scores = Path(f"results/{metric_name}_scores/")
path_to_scores.mkdir(exist_ok=True)

torch.save({model_name : model_to_mean_std[model_name]},
           path_to_scores/f"{model_name}_mean_std.pt")

/scratch/mz2476/miniconda3/envs/my_base/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


KeyError: 'val_hi'

In [76]:
# for model_name in model_to_mean_std.keys():
#     torch.save({model_name : model_to_mean_std[model_name]}, f"results/f1_micro_scores/{model_name}_mean_std.pt")

In [108]:
# Load
model_to_mean_std = {}
path_to_scores = Path(f"results/{metric_name}_scores/")
[model_to_mean_std.update(torch.load(fname)) for fname in path_to_scores.iterdir()]

mean_mk = np.array([mean_std[0] for mean_std in model_to_mean_std.values()])
std_mk  = np.array([mean_std[1] for mean_std in model_to_mean_std.values()])

axis = plot_errorbars_by_model(np.array(mean_mk), np.array(std_mk), labels_m=model_to_mean_std.keys())

[None, None, None]

/scratch/mz2476/miniconda3/envs/my_base/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/mz2476/miniconda3/envs/my_base/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


,precision_macro,recall_macro,f1_macro,precision_micro,recall_micro,f1_micro
0,0.601943,0.474778,0.501424,0.806897,0.659155,0.725581
1,0.581520,0.475961,0.501708,0.798276,0.685926,0.737849
2,0.601665,0.457821,0.497679,0.795041,0.668056,0.726038


In [69]:
[np.arange(start, 100, num_splits) for start in range(num_splits)]

[array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80,
        85, 90, 95]),
 array([ 1,  6, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56, 61, 66, 71, 76, 81,
        86, 91, 96]),
 array([ 2,  7, 12, 17, 22, 27, 32, 37, 42, 47, 52, 57, 62, 67, 72, 77, 82,
        87, 92, 97]),
 array([ 3,  8, 13, 18, 23, 28, 33, 38, 43, 48, 53, 58, 63, 68, 73, 78, 83,
        88, 93, 98]),
 array([ 4,  9, 14, 19, 24, 29, 34, 39, 44, 49, 54, 59, 64, 69, 74, 79, 84,
        89, 94, 99])]

In [64]:
indices = np.arange(500)
indices = np.arange(500, 1000)

dict_of_metrics = learner.get_test_metrics(
    data.Subset(learner.val_loader.dataset, indices), device=learner.device)

import pandas as pd

pd.DataFrame([dict_of_metrics, dict_of_metrics])

/scratch/mz2476/miniconda3/envs/my_base/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/mz2476/miniconda3/envs/my_base/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


,precision_macro,recall_macro,f1_macro,precision_micro,recall_micro,f1_micro
0,0.651972,0.482795,0.534985,0.798883,0.680304,0.734841
1,0.651972,0.482795,0.534985,0.798883,0.680304,0.734841


In [ ]:
# train the model
model_name = "mixed_en_hi_ru_" + "_".join([str(key) + "_" + str(value) for key, value in result.items()])
print(model_name)
metrics_dict = train_model(train_loader, val_loader, model, criterion, optimizer, options, device,
                num_epochs=10, model_name="model", save_model=False)
result.update(metrics_dict)

In [8]:
optimizer.param_groups[0]["lr"]

0.01

In [19]:
# loaders = create_data_loaders_for_model(wiki_loaders["train"], wiki_loaders["val"])
# # create dataloader
# wiki_loaders = {}


# for split, wiki_dataset in dict_wiki_tensor_dataset.items():
#     wiki_loaders[split] = DataLoader(
#         wiki_dataset, 
#         batch_size=batch_size, 
#         shuffle=True, 
#         collate_fn=partial(pad_collate_fn, word_to_index=word_to_index)
#     )


# train_model(
#     wiki_loaders, model, criterion, optimizer, options=options, num_epochs=num_epochs, 
#     model_name=model_name, save_model=SAVE_MODEL
# )

# results_df = results_df.append(result, ignore_index=True)
#     results_df.to_csv("results/results_tuning_2_3_layers_maxlen_500.csv"


 {'optimizer': 'Adam', 'num_hidden': 2, 'dim_hidden': 150, 'dropout_rate': 0.2, 'learning_rate': 0.01, 'num_epochs': 15}


RuntimeError: CUDA out of memory. Tried to allocate 2.28 GiB (GPU 0; 7.93 GiB total capacity; 6.85 GiB already allocated; 626.56 MiB free; 6.49 MiB cached)